In [24]:
import errant
import pandas as pd
import difflib

In [253]:
def get_diff(a,b):
    s= [li for li in difflib.ndiff(a, b) if li[0] != ' ']
    return s
    

In [254]:
get_diff('evolve','evolving')

['- e', '+ i', '+ n', '+ g']

In [255]:
punc=['- .','+ .','- ,','+ ,',"- '","+ '", '- ;','+ ;','- -','+ -','-  ','+  ']

In [256]:
ing = ['+ i', '+ n', '+ g']

In [257]:
set(ing).issubset(set(get_diff('evolve','evolving')))

True

In [258]:
def error_type(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    for e in edits:
        if 'DET' in e.type:
            return 'Articles'
        elif 'PREP' in e.type or 'PART' in e.type:
            return 'Preposition'
        elif 'PUNCT' in e.type or get_diff(e.o_str, e.c_str)[0] in punc:
            return 'Punctuation'
        elif 'VERB' in e.type or set(ing).issubset(set(get_diff(e.o_str, e.c_str))):
            if 'SVA' in e.type:
                return 'Subject Verb Agreement'
            else:
                return 'Verb Form'
        elif 'NOUN' in e.type or 'ADJ' in e.type or 'MORPH' in e.type or 'SPELL' in e.type or 'ORTH' in e.type:
            return 'Word Form'
        else:
            return 'Other'

In [178]:
def get_action(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    for e in edits:
        if 'R:' in e.type:
            return 'replace'
        elif 'M:' in e.type:
            return 'insert'
        elif 'U:' in e.type:
            return 'remove'
            
            

In [179]:
def get_expla(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    if error_type(s1,s2)=='Verb Form':
        for e in edits:
            if 'TENSE' in e.type:
                return 'Verb Tense Error'
            elif 'FORM' in e.type:
                return 'Verb Form error'
            elif set(ing).issubset(set(get_diff(e.o_str, e.c_str))):
                return 'Present continuous tense'
            else:
                return 'Other verb error'
    elif error_type(s1,s2)=='Word Form':
        for e in edits:
            if 'NUM' in e.type:
                return 'Noun Number error'
            elif 'ADJ' in e.type:
                return 'Adjective error'
            elif 'MORPH' in e.type:
                return 'Morphology error'
            elif 'ORTH' in e.type:
                return 'Orthography error'
            else:
                return 'Other word form error'
            
        
        

In [9]:
data = pd.read_csv('ErrorData.csv')

In [99]:
data['give_action'] = data.apply(lambda x: get_action(x.question, x.solution),axis=1)

In [259]:
data['give_category'] = data.apply(lambda x: error_type(x.question, x.solution),axis=1)

In [100]:
data.head(30)

,questionid,category,round,question,solution,explanation,type,action,give_category,give_action
0,1,Articles,1,These devices generate huge amount of data tha...,These devices generate a huge amount of data t...,Article required,C,insert,Articles,insert
1,1,Articles,1,Fashion items make up large portion of e-comme...,Fashion items make up a large portion of e-com...,"The word ""portion"" is a single, countable noun...",SP,insert,Articles,insert
2,1,Articles,2,"Firstly, increasing number of people are reviv...","Firstly, an increasing number of people are re...",Article required,C,insert,Articles,insert
3,1,Articles,3,"In the recent years, the rise of technology in...","In recent years, the rise of technology in the...",No article required,C,remove,Articles,remove
4,1,Articles,4,"The Sugar Detox Diet, which aims to promote a ...","The Sugar Detox Diet, which aims to promote he...",No article required,C,remove,Articles,remove
5,1,Preposition,1,A study by ABI Research (2017) forecasted that...,A study by ABI Research (2017) forecasted that...,Idiomatic expressions: will grow by and will g...,C,insert,Preposition,insert
6,1,Preposition,1,One method to measure brand positioning would ...,One method to measure brand positioning would ...,"Modals or modal auxiliary verbs (can, could, m...",SP,insert,Verb Form,insert
7,1,Preposition,2,I have researched on recent health food trends.,I have researched recent health food trends.,No preposition required after research as a verb,C,remove,Preposition,remove
8,1,Preposition,3,Board games help people to build up social ski...,Board games help people to build social skills...,No preposition required because the direction ...,C,remove,Preposition,remove
9,1,Preposition,4,The cost of a mobile phone can amount from $1000.,The cost of a mobile phone can amount to $1000.,"Amount to means up to an amount, as in this case.",C,replace,Preposition,replace


In [260]:
data['compare']=data.apply(lambda x: x['category']==x['give_category'], axis=1)
data['compare'].value_counts()

True     312
False     18
Name: compare, dtype: int64

In [261]:
wrong=data.loc[data['compare']==False]
wrong

,questionid,category,round,question,solution,explanation,type,action,give_category,give_action,compare
6,1,Preposition,1,One method to measure brand positioning would ...,One method to measure brand positioning would ...,"Modals or modal auxiliary verbs (can, could, m...",SP,insert,Verb Form,insert,False
49,2,Subject Verb Agreement,3,Even as viewership soar for new competitors li...,Even as viewership soars for new competitors l...,"""viewership"" takes the singular form of the verb.",C,replace,Word Form,replace,False
50,2,Subject Verb Agreement,4,These sources prove that social media sites li...,These sources prove that social media sites li...,"""social media sites"" takes the plural form of ...",C,replace,Word Form,replace,False
116,4,Verb Form,3,"According to Cheah (2017) and Nardoci (2012), ...","According to Cheah (2017) and Nardoci (2012), ...",Present perfect tense. The subject is 'growth'...,C,replace,Subject Verb Agreement,replace,False
117,4,Verb Form,4,Another trend we should explore be digital det...,Another trend we should explore is digital det...,Present simple tense,C,replace,Subject Verb Agreement,replace,False
123,4,Word Form,4,"According to statistics, there is an uprising ...","According to statistics, there is an rising tr...","""An Uprising"" is a noun that means rebellion. ...",C,replace,Verb Form,replace,False
129,5,Preposition,1,"Therefore, it is vital for Mediacorp grow the ...","Therefore, it is vital for Mediacorp to grow t...",It's vital for someone to do something; it's v...,C,insert,Verb Form,insert,False
138,5,Punctuation,4,The articles provided a different perspective ...,The articles provided a different perspective ...,millennials is a common noun and does not have...,C,replace,Word Form,replace,False
143,5,Subject Verb Agreement,4,"Research has shown that music, yoga and guided...","Research has shown that music, yoga and guided...","""music, yoga and guided imagery"" take the plur...",C,replace,Word Form,replace,False
145,5,Verb Form,1,Mental health affects students' grades or lead...,Mental health affects students' grades or lead...,Simple present tense should be used here. The ...,SP,replace,Subject Verb Agreement,replace,False


In [130]:
wrong.to_csv('wrong.csv')

In [110]:
s1 = 'It is forecasted that AR users will increase by another 35% over the next 5 years and millennials play a large part in that (Madigan, 2016).'
s2 = 'It is forecasted that AR users will increase by another 35% over the next 5 years and millennials will play a large part in that (Madigan, 2016).'



In [111]:
get_expla(s1,s2)

'Verb Tense Error'

In [105]:
s3 = 'The rising incidences of stroke among young people is an issue of concern.'
s4 = 'The rising incidence of stroke among young people is an issue of concern.'

In [109]:
get_expla(s3,s4)

'Noun Number error'

In [195]:
data.loc[72]['question']

'It is not easy to customize the online content to suit consumers interests.'

In [265]:
s5_1='The cost of seeing a therapist can amount from $75-$150.'
s5_2 = 'The cost of seeing a therapist can range from $75-$150.'

In [266]:
error_type(s5_1,s5_2)

'Verb Form'

In [267]:
annotator = errant.load('en')
orig = annotator.parse(s5_1)
cor = annotator.parse(s5_2)
edits = annotator.annotate(orig, cor)
for e in edits:
    print(e.o_start, e.o_end, e.o_str, e.c_start, e.c_end, e.c_str, e.type)
    la=get_diff(e.o_str, e.c_str)

7 8 amount 7 8 range R:VERB
